# 분류의 성능 평가 지표
- 정확도(Accuracy)
- 오차행렬(Confusion Matrix)
- 정밀도(Precision)
- 재현율(Recall)
- F1 Score
- ROC AUC

> 분류는 결정 클래스 값 종류의 유형에 따라 긍정/부정의 두개의 결과만 가지는 이진 분류와 여러개의 결정 클래스를 가지는 멀티 분류 두 가지로 나뉜다

## 정확도
- 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
- 정확도를 평가 지표로 사용할 땐 매우 신중해야함. 특히나 불균형한 레이블 값 분포에서는 적절하지 않음

In [14]:
# 잘못된 예제 - 타이타닉 건수를 가지고 성별 1이면 0, 아니면 1로 예측
import numpy as np
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit( ) 메소드는 아무것도 학습하지 않음. 
    def fit(self , X , y=None):
            pass
    
    # predict( ) 메소드는 단순히 Sex feature가 1 이면 0 , 그렇지 않으면 1 로 예측함. 
    def predict(self, X):
        pred = np.zeros( ( X.shape[0] , 1))
        for i in range (X.shape[0]) :
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        
        return pred

In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습데이터/테스트 데이터 분할. 
titanic_df = pd.read_csv('./titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, \
                                                  test_size=0.2, random_state=0)

# 위에서 생성한 Dummy Classifier를 이용하여 학습/예측/평가 수행. 
myclf = MyDummyClassifier()
myclf.fit(X_train ,y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test , mypredictions)))

Dummy Classifier의 정확도는: 0.7877


## 오차 행렬
- 이진 분류에서 성능 지표로 많이 활용됨
- 학습된 분류 모델이 예측을 수행하면서 얼마나 혼동중인지 보여주는 지표. 이진분류의 예측 오류가 얼마인지 / 어떤 유형의 예측 오류가 발생하는지
- TN, FP, FN, TP의 4분면 행렬에서 실제 레이블 클래스 값과 예측 레이블 클래스 값이 어떻게 매핑되는지 나타냄
- TN, FP, FN, TP 값을 조합해 Classifier의 성능을 측정할 수 있는 주요 지표인 정확도, 정밀도, 재현율 값을 알 수 있다
    - 정확도: 예측 결과와 실제값이 동일한 건수 / 전체 데이터의 수 = (TN + TP) / total

## 정밀도와 재현율
- Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가 지표
- 재현율과 정밀도 모두 TP를 올리는데 초점을 두지만 재현율은 FN을 낮추는데에, 정밀도는 FP를 낮추는데 초점을 낮춤. 이런 특성때문에 이 둘은 서로 보완적인 지표로 적용

### 정밀도
- TP / (FP + TP)
- 예측을 positive로 한 대상중 예측과 실제 값이 positive로 일치한 데이터의 비율을 뜻함. 양성 예측도
- 정밀도가 중요지표인 경우: 실제 negative 양성 데이터를 positive로 판단시 업무상 큰 영향이 발생하는 경우. ex)스팸 메일
- 사이킷런의 precision_socre()

### 재현율
- TP / (FN + TP)
- 실제 값이 positive인 대상 중에 예측과 실제 값이 positive로 일치한 데이터의 비용을 뜻함. 민감도나 TPR로 불림
- 재현율이 중요지표인 경우: 실제 positive 양성 데이터를 negative로 판단시 업무상 큰 영향이 발생하는 경우 ex)암 판단, 보험 사기
- 사이킷런의 recall_score()

In [19]:
from sklearn.metrics import accuracy_score, precision_score , recall_score , confusion_matrix

def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy , precision ,recall))

In [21]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression

# 원본 데이터를 재로딩, 데이터 가공, 학습데이터/테스트 데이터 분할. 
titanic_df = pd.read_csv('./titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df= titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, \
                                                    test_size=0.20, random_state=11)

lr_clf = LogisticRegression()

lr_clf.fit(X_train , y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test , pred)

오차 행렬
[[104  14]
 [ 13  48]]
정확도: 0.8492, 정밀도: 0.7742, 재현율: 0.7869


/Users/prograsshopper/.pyenv/versions/3.6.4/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### 정밀도/재현율 트레이드오프
- 분류하는 업무 특성상 정밀도나 재현율중 특별히 강조되어야하는게 있다면 결정 임곗값을 조정해 둘 중 하나의 수치를 올릴 수 있음